In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rasterio
from rasterio.plot import show
import os
import pandas as pd
import xarray as xr
import glob
import numpy as np
import sqlite3
import plotly.express as px
from geopy.distance import geodesic
import matplotlib.colors as mcolors

from Functions_sfcwinds import *



In [3]:
# Load HRRR files

HRRR_folder = "/kfs2/projects/sfcwinds/HRRR"
area = "US_SW"
date_string = "2024-05-06"

file_pattern = os.path.join(HRRR_folder, f"hrrr_{area}_{date_string[:]}*.nc")
file_list = sorted(glob.glob(file_pattern))
if file_list:
    hrrr = xr.open_mfdataset(file_list[:], concat_dim ="valid_time",combine='nested', chunks="auto", parallel = True)

hrrr['longitude'] = xr.where(hrrr['longitude'] > 180, hrrr['longitude'] - 360, hrrr['longitude'])

# test hourly vs subourly wind speed
# plt.figure()
# plt.plot(hrrr.valid_time.values, hrrr.isel(x=100, y=100).u10_h.values, ".", color="blue", label = "HRRR sfc")    
# plt.plot(hrrr.valid_time.values, hrrr.isel(x=100, y=100).u10.values, ".", color="orange", label = "HRRR subh") 
# plt.plot(hrrr.valid_time.values, hrrr.isel(x=100, y=100).v10_h.values, "x", color="blue", label = "HRRR sfc")    
# plt.plot(hrrr.valid_time.values, hrrr.isel(x=100, y=100).v10.values, "x", color="orange", label = "HRRR subh") 

hrrr["wspd10"] = (hrrr.u10**2 + hrrr.v10**2)**0.5

In [4]:
hrrr

<xarray.Dataset>
Dimensions:            (isobaricInhPa: 3, valid_time: 96, y: 750, x: 870)
Coordinates:
  * isobaricInhPa      (isobaricInhPa) float64 850.0 500.0 250.0
  * valid_time         (valid_time) datetime64[ns] 2024-05-06T00:15:00 ... 20...
Dimensions without coordinates: y, x
Data variables: (12/25)
    step               (valid_time) timedelta64[ns] 02:00:00 ... 02:00:00
    latitude           (valid_time, y, x) float64 dask.array<chunksize=(96, 750, 870), meta=np.ndarray>
    longitude          (valid_time, y, x) float64 dask.array<chunksize=(96, 750, 870), meta=np.ndarray>
    u                  (valid_time, isobaricInhPa, y, x) float32 dask.array<chunksize=(76, 1, 615, 714), meta=np.ndarray>
    v                  (valid_time, isobaricInhPa, y, x) float32 dask.array<chunksize=(76, 1, 615, 714), meta=np.ndarray>
    t2m                (valid_time, y, x) float32 dask.array<chunksize=(77, 612, 710), meta=np.ndarray>
    ...                 ...
    tcc                (valid_time, y, x) float32 dask.array<chunksize=(77, 612, 710), meta=np.ndarray>
    cape               (valid_time, y, x) float32 dask.array<chunksize=(77, 612, 710), meta=np.ndarray>
    heightAboveGround  (valid_time) float64 10.0 10.0 10.0 ... 10.0 10.0 10.0
    u10                (valid_time, y, x) float32 dask.array<chunksize=(77, 612, 710), meta=np.ndarray>
    v10                (valid_time, y, x) float32 dask.array<chunksize=(77, 612, 710), meta=np.ndarray>
    wspd10             (valid_time, y, x) float32 dask.array<chunksize=(77, 612, 710), meta=np.ndarray>

In [5]:
# Read obs data

# Read metadata to filter what we want
meta = pd.read_csv("/kfs2/projects/sfcwinds/observations/metadata_CONUS.csv")

# filter which data to read
base_dir = "/kfs2/projects/sfcwinds/observations/"
stations = meta[meta.state == "NM"].station_id.values
networks = ["*"]
years    = [date_string[:4]]

#Read observations according to filter
years = [str(y) for y in years]
all_dfs = []

for net in networks:
    for station in stations:
        for year in years:
            pattern = os.path.join(base_dir, net, station, f"{year}.parquet")
            found = glob.glob(pattern)

            for f in found:
                try:
                    df = pd.read_parquet(f)
                    # Extract station_id from the path (e.g., .../CoAgMet/HOT01/2023.parquet)
                    station_id = os.path.basename(os.path.dirname(f))  # 'HOT01'
                    df["station_id"] = station_id
                    all_dfs.append(df)
                except Exception as e:
                    print(f"Failed to read {f}: {e}")

if not all_dfs:
    print("No files found.")
else:
    print(f"{len(all_dfs)} files found.")
    obs = pd.concat(all_dfs, ignore_index=True)
    obs["timestamp"] = obs["timestamp"].dt.tz_convert(None)

# Merge with metadata
obs = obs.merge(meta[["station_id", "lat", "lon", "height", "elev", "source_network", "state"]], on="station_id", how="left")

144 files found.


In [ ]:

plt.figure()
for stid  in obs.station_id.unique()[:]:

    # Time series of each station
    station_obs = obs[obs.station_id == stid]

    if len(station_obs.dropna(subset = "windspeed")) > 10:

        # resample to 15 min
        station_obs = station_obs.resample("15min", on="timestamp").mean(numeric_only=True)

        # Get closest HRRR grid point
        hrrr_loc = find_closest_HRRR_loc(hrrr, [np.nanmean(station_obs.lat.unique()), np.nanmean(station_obs.lon.unique())])

        # MErge Obs and HRRR into one dataframe
        merged = station_obs.merge(hrrr_loc.drop_dims("isobaricInhPa").to_dataframe(),
                                   left_on = "timestamp",
                                   right_index = True,
                                   how = "inner")

        # Plot Obs vs. HRRR
        plt.plot(merged.windspeed, merged.u10,".")
    

In [ ]:
### Plot surface roughness

hrrr_plot = hrrr.sel(valid_time=pd.to_datetime(date_string), method = "nearest")

# Create a figure with a map projection
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
# plt.title(f"HRRR day: {hrrr_plot.time.values} \n Metar day: {metar_time}")

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle='-', linewidth=0.5, edgecolor="black") 
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.3)
ax.add_feature(cfeature.LAKES, edgecolor='black', alpha=0.3)
ax.add_feature(cfeature.RIVERS, edgecolor='blue', alpha=0.3)

gl = ax.gridlines(draw_labels=True, linestyle="-", linewidth=0.5, alpha=0.0)
gl.right_labels = False  # Hide right labels
gl.top_labels = False  # Hide top labels

ax.set_extent([-110,-102, 31.5, 37.5], crs=ccrs.PlateCarree())  

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# HRRR data
sc = contour = plt.pcolormesh(hrrr_plot.longitude, 
                         hrrr_plot.latitude, 
                         hrrr.fsr.dropna(dim="valid_time").isel(valid_time=0).squeeze(), cmap="gnuplot", shading="auto", transform=ccrs.PlateCarree())

# Sandia
ax.plot(Sandia_coords[1], Sandia_coords[0],transform=ccrs.PlateCarree(), marker='o', ms=10, color='black',  alpha=0.9)

# Add colorbar
cbar = plt.colorbar(sc, ax=ax, orientation='vertical', label="Surface Roughness (m)")

# Labels and title
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
#ax.set_title("Surface roughness")

In [48]:
fsr_data.min().values

array(1.e-04, dtype=float32)